In [1]:
from torchsummary import summary
from datetime import datetime
import torch.nn as nn
import torch
import random
import numpy as np

from src.nn.regression_dataset import RegressionDataset
from src.nn.to_tensor import ToTensor
from src.nn.create_data_loaders import create_data_loaders
from src.nn.cnn_regressor import CNNRegressor
from src.nn.training import training
from src.nn.plot_losses import plot_losses
from src.nn.weighted_mse_loss import WeightedMSELoss
import src.ctes.num_ctes as nctes
import src.ctes.path_ctes as pctes

In [ ]:
# An epoch takes around 3 minutes (train + val) on a school computer with GPUs
# Call several times the following function to start several trainings 

In [10]:
def training_general(data_path, batch_size, valid_size, test_size, criterion, model_name, n_epochs, dropout_probs = None, lr = 0.001, weight_decay = 0):
    seed = 42
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    data_path = data_path 
    transform = ToTensor()
    sample_size = nctes.LEN_SAMPLE

    print("Getting data path...")
    data = RegressionDataset(data_path, transform, sample_size)
    batch_size = batch_size
    valid_size = valid_size
    test_size = test_size
    print("Data path saved.")
    print("")
    
    print("Checking GPU availability...")
    train_loader, valid_loader, test_loader = create_data_loaders(batch_size, valid_size, test_size, data)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device ' + str(device))
    print("")
    sample_size = nctes.LEN_SAMPLE

    # model params 
    dropout_probs = dropout_probs
    model = CNNRegressor(input_size=sample_size, dropout_probs=dropout_probs)
    model.to(device=device)
    criterion = criterion
    model_name = model_name
    params = model.parameters()
    lr = lr
    weight_decay = weight_decay
    n_epochs = n_epochs

    
    optimizer = torch.optim.Adam(params, lr, weight_decay = weight_decay)

    print("Saving model parameters...")
    timestamp = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
    hyperparams_path = f"../../data/hyperparams_{timestamp}.npz"
    model_path = f"../../data/model_{timestamp}.pt"
    losses_path = f"../../data/losses_{timestamp}.npz"
    np.savez(hyperparams_path, 
             len=len(data), 
             test_size=test_size, 
             valid_size=valid_size, 
             epochs=n_epochs, 
             batch_size=batch_size, 
             criterion=str(criterion), 
             optimizer=str(optimizer), 
             lr=lr,
             seed=seed,
             weight_decay = weight_decay,
             dropout_probs = dropout_probs, 
             model_name = model_name)
    print("Model saved.")
    print("")
    
    train_losses, valid_losses = training(n_epochs, train_loader, valid_loader, model, criterion, optimizer, device, model_path)

    print("Saving losses...")
    np.savez(losses_path , train=np.array(train_losses), val=np.array(valid_losses))
    print("Losses saved.")

    return train_losses, valid_losses


In [ ]:
data_path = pctes.DATAPATH_fm
batch_size = 6
valid_size = 0.2
test_size = 0.2
criterion = nn.MSELoss()
dropout_probs=[0.7, 0.7, 0.7, 0.7, 0.6, 0.6, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.7, 0.7]
model_name = "Variable dropout probas and L2 regularization" # name given to the trained model
lr = 0.001
weight_decay = 1e-5 # for L2 regularization
n_epochs = 2

train_losses, valid_losses,  = training_general(data_path, batch_size, valid_size, test_size, criterion, model_name, n_epochs, dropout_probs, lr, weight_decay)

Getting data path...
Data path saved.

Checking GPU availability...
Using device cuda

Saving model parameters...
Model saved.

Epoch 1/2 ...
Training ...


100%|████████████████████████████████████████████████████████████████████████████████████| 1366/1366 [02:37<00:00,  8.65it/s]


Validating ...


100%|██████████████████████████████████████████████████████████████████████████████████████| 342/342 [00:13<00:00, 26.08it/s]


Epoch: 1 	training Loss: 0.191912 	validation Loss: 0.131715
Validation loss decreased (inf --> 0.131715).  Saving model ...
Epoch 2/2 ...
Training ...


 94%|███████████████████████████████████████████████████████████████████████████████▏    | 1288/1366 [02:34<00:09,  8.36it/s]

In [ ]:
plot_losses(len(train_losses), [train_losses, valid_losses], ["Train", "Val"])